In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%pylab inline
import pandas as pd
import json
import seaborn
import copy
import os
from Chapter2.CreateDataset import CreateDataset
from util.VisualizeDataset import VisualizeDataset
from Chapter3.OutlierDetection import DistributionBasedOutlierDetection
from Chapter3.OutlierDetection import DistanceBasedOutlierDetection
from util.VisualizeDataset import VisualizeDataset
from util import util
from util.parser import get_labels,generate_csv,generate_labels,get_sensor_values
import time

Populating the interactive namespace from numpy and matplotlib


/home/cts/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Below cell has global variables (path-names mostly)

In [3]:
dataset_folder = './datasets'
files = os.listdir(dataset_folder)
print(files)
dataset_fname  = 'log_assignment2'
event_fname = 'log_events_assignment2'
result_dataset_path = './intermediate_datafiles/'
csv_dataset_path = os.path.join(dataset_folder,'csv/')
dataset_path = os.path.join(dataset_folder,dataset_fname)
event_path = os.path.join(dataset_folder,event_fname)
output_fname = 'dataset.csv'

['log_assignment2', 'csv', 'log_events_assignment2']


In [4]:
if not os.path.exists(result_dataset_path):
    print('Creating result directory: ' + result_dataset_path)
    os.makedirs(result_dataset_path)
if not os.path.exists(csv_dataset_path):
    print('Creating result directory: ' + csv_dataset_path)
    os.makedirs(csv_dataset_path)

### LOG PARSING FUNCTION CALL

In [5]:
labels = get_labels(dataset_path)
print('Label Mappings:\n')
print(json.dumps(labels,indent=2))


Label Mappings:

{
  "standing": [
    6, 
    11
  ], 
  "walking": [
    9, 
    12
  ], 
  "sitting": [
    10, 
    13
  ], 
  "stairsdown": [
    4, 
    7
  ], 
  "stairsup": [
    5, 
    8
  ], 
  "ondesk": [
    14
  ]
}


In [6]:
sensors_dict = get_sensor_values(dataset_path)
print('Sensors in log file')
sensors = sensors_dict.keys()
print(json.dumps(sensors,indent=2))

Sensors in log file
[
  "bmi160 accelerometer", 
  "bmi160 gyroscope", 
  "rotation vector"
]


# CSV generation cell

In [7]:
for sensor in sensors:
    if sensor == 'rotation vector':
        generate_csv(sensors_dict[sensor],
                     header=['x','y','z','theta','phi','timestamp'],
                     fname = os.path.join(csv_dataset_path,sensor+'.csv'))
    else:
        generate_csv(sensors_dict[sensor],fname = os.path.join(csv_dataset_path,sensor+'.csv'))
generate_labels(fname=event_path,log_fname=dataset_path,fout=csv_dataset_path+'labels.csv');

# Dataset creation cell

In [8]:
granularities = [250,1000,10000]
datasets = []


for milliseconds_per_instance in granularities:
    initial = time.time()
    # Create an initial dataset object with the base directory for our data and a granularity
    DataSet = CreateDataset(csv_dataset_path, milliseconds_per_instance)
    print('granularity is %d ms'%milliseconds_per_instance)
    
    DataSet.add_numerical_dataset('bmi160 accelerometer.csv','timestamp',['x','y','z'],'avg','acc_phone_')
    DataSet.add_numerical_dataset('bmi160 gyroscope.csv','timestamp',['x','y','z'],'avg','gyr_phone_')
    DataSet.add_numerical_dataset('rotation vector.csv','timestamp',['x','y','z','theta','phi'],'avg','rotation_phone_')
    DataSet.add_event_dataset('labels.csv','label_start','label_end','label','binary')
    print('Added all sensors and labels')
    dataset = DataSet.data_table
#     util.print_statistics(dataset)
    datasets.append(copy.deepcopy(dataset))
    dataset.to_csv(result_dataset_path + '/'+str(milliseconds_per_instance)+ '_dataset.csv')
    print('Dataset created,\tDuration:%.5f seconds'%(time.time()-initial))
    print('====================================')
    
    

granularity is 250 ms
Added all sensors and labels
Dataset created,	Duration:18.89590 seconds
granularity is 1000 ms
Added all sensors and labels
Dataset created,	Duration:5.60809 seconds
granularity is 10000 ms
Added all sensors and labels
Dataset created,	Duration:1.76530 seconds
